In [4]:
#Fast feature detection

import cv2
import numpy as np

input_image = cv2.imread('lena.png')
gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
# Version under opencv 3.0.0 cv2.FastFeatureDetector()
fast = cv2.FastFeatureDetector_create()

# Detect keypoints
keypoints = fast.detect(gray_image, True)
print("Number of keypoints with non max suppression:", len(keypoints))
# Draw keypoints on top of the input image
img_keypoints_with_nonmax=input_image.copy()
cv2.drawKeypoints(input_image, keypoints, img_keypoints_with_nonmax, color=(0,255,0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) 
cv2.imshow('FAST keypoints - with non max suppression', img_keypoints_with_nonmax)
 
# Disable nonmaxSuppression
fast.setNonmaxSuppression(False)
# Detect keypoints again
keypoints = fast.detect(gray_image, False)
print("Total Keypoints without nonmaxSuppression:", len(keypoints))
# Draw keypoints on top of the input image
img_keypoints_without_nonmax=input_image.copy()
cv2.drawKeypoints(input_image, keypoints, img_keypoints_without_nonmax, color=(0,255,0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('FAST keypoints - without non max suppression', img_keypoints_without_nonmax)

cv2.waitKey()
cv2.destroyAllWindows()

Number of keypoints with non max suppression: 1356
Total Keypoints without nonmaxSuppression: 5705


In [1]:
# SIFT feature detection
# need rebuild opencv contrib module from source

import cv2
import numpy as np
input_image = cv2.imread('sakura.jpg')
gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
# For version opencv < 3.0.0, use cv2.SIFT()
sift = cv2.SIFT_create()
keypoints = sift.detect(gray_image, None)
# cv2.drawKeypoints(input_image, keypoints, input_image, keypoints,flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
output_image = cv2.drawKeypoints(input_image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('SIFT features', output_image)
cv2.waitKey()
cv2.destroyAllWindows()

In [3]:
#template matching
import cv2
import numpy as np

img = cv2.imread('red_blood_cell.jpg', cv2.IMREAD_COLOR)
height, width = img.shape[0:2]
print(img.shape)
show_img = np.copy(img)
mouse_pressed = False
y0 = x0 = w = h = 0

def mouse_callback(event, _x, _y, flags, param):
    global show_img, x0, y0, w, h, mouse_pressed
    if event == cv2.EVENT_LBUTTONDOWN:
        mouse_pressed = True
        x0, y0 = _x, _y
        show_img = np.copy(img)
    elif event == cv2.EVENT_MOUSEMOVE:
        if mouse_pressed:
            show_img = np.copy(img)
            cv2.rectangle(show_img, (x0, y0), (_x, _y), (0, 255, 0), 2)
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_pressed = False
        w, h = _x - x0, _y - y0
        
cv2.namedWindow('image')
cv2.setMouseCallback('image', mouse_callback)

#Acquire a template through mouse dragging
while True:
    cv2.imshow('image', show_img)
    k = cv2.waitKey(1)
    #chr(97) => 'a'
    #ord('a') => 97
    if k == ord('a') and not mouse_pressed:
        if w*h > 0:
            break
cv2.destroyAllWindows()

template = np.copy(img[y0:y0+h, x0:x0+w])
print(template.shape)
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
show_img = np.copy(img)

#Perform template matching with various methods (0~5)
while True:
    cv2.imshow('tm', show_img)
    k = cv2.waitKey()
    if k == 27:
        break
    elif k > 0 and chr(k).isdigit():
        index = int(chr(k))
        if 0 <= index < len(methods):
            method = methods[index]
            res = cv2.matchTemplate(img, template, eval(method))
            print('min:{} max:{}'.format(np.min(res), np.max(res)))
            res = cv2.normalize(res, None, 0, 1, cv2.NORM_MINMAX)            
            print('min_n:{} max_n:{}'.format(np.min(res), np.max(res)))
            #if index >= methods.index('cv2.TM_SQDIFF'):
            #    loc = np.where(res < 0.2)
            #else:
            #    loc = np.where(res > 0.8)            
            #for pt in zip(*loc[::-1]):
            #    cv2.rectangle(res, (pt[0] - int(w/2), pt[1] - int(h/2)), (pt[0] + int(w/2), pt[1] + int(h/2)),(0, 0, 0), 1)
            res_img = np.copy(res)
            for y in range(0, res.shape[0]):
                for x in range(0, res.shape[1]):
                    if (index >= methods.index('cv2.TM_SQDIFF')) and (res[y,x] < 0.2):            
                        cv2.rectangle(res_img, (x - int(w/2), y - int(h/2)), (x + int(w/2), y + int(h/2)),(0, 0, 0), 1)
                    elif (index < methods.index('cv2.TM_SQDIFF')) and (res[y,x] > 0.8):            
                        cv2.rectangle(res_img, (x - int(w/2), y - int(h/2)), (x + int(w/2), y + int(h/2)),(0, 0, 0), 1)
                       
            #print(res_img.shape)
            res_img = cv2.resize(res_img, (width, height))*255
            res_img = cv2.cvtColor(res_img, cv2.COLOR_GRAY2BGR).astype(np.uint8)
            #print(res_img.shape)
            cv2.putText(res_img, method, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
            show_img = np.copy(img)
            show_img = np.hstack((show_img, res_img))
cv2.destroyAllWindows()

(238, 312, 3)
(30, 29, 3)


###### %%writefile test
## Practice
1. Input images from video file WiiPlay.mp4 with frame number between 4820 and 5000 (i.e., level 15).
2. Acquire a face template from the first frame (frame number = 4820).
3. Use <i>cv2.matchTemplate()</i> to perform template matching on subsequent frames.
4. Draw a blue rectangle around each good match and show the output images.
5. (Optional) Use <i>cv2.minMaxLoc()</i> to find the best match and draw a red rectangle around the best match.
6. Observe which method is better ('cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED') 
7. Upload your Jupyter code file (*.ipynb)

In [10]:
print (int(5.8))

5
